# 1. Getting started with GEMs and FBA

## Authors
* Arianna Basile, MRC Toxicology Unit, University of Cambridge
* Francisco Zorrilla, MRC Toxicology Unit, University of Cambridge

## Learning outcomes

In this tutorial you will use [cobrapy](https://cobrapy.readthedocs.io/en/latest/) to learn the following:

* **1.1**: Import a metabolic reconstruction
* **1.2**: Inspect the reactions of your model
* **1.3**: Inspect the metabolites in your model
* **1.4**: Inspect the genes in your model
* **1.5**: Run flux balance analysis (FBA)
* **1.6**: Perform in-silico gene knockout experiments

## Setup

In [ ]:
# Import required packages
import cobra

# Enable autocompleting with tab key
%config Completer.use_jedi = False

## 1.1 Import a reconstruction

The [Systems Biology Markup Language](https://sbml.org/) is an XML-based standard format for distributing models which has support for COBRA models through the FBC extension version 2.

Cobrapy has native support for reading and writing SBML with FBCv2. Please note that all id’s in the model must conform to the SBML SID requirements in order to generate a valid SBML file.

Let's download and import the model of <a href="http://bigg.ucsd.edu/search?query=Saccharomyces+cerevisiae+S288C"> Saccharomyces cerevisiae</a>.

In [ ]:
model_yeast=cobra.io.read_sbml_model("iMM904.xml.gz")

The reactions, metabolites, genes, and compartments attributes of the cobrapy model are a special type of `list` called a `cobra.DictList`, and each one is made up of `cobra.Reaction`, `cobra.Metabolite`, `cobra.Gene`, `cobra.Compartment` objects, respectively.

In [ ]:
print("Reactions: ",len(model_yeast.reactions))
print("Metabolites: ",len(model_yeast.metabolites))
print("Genes: ",len(model_yeast.genes))
print("Compartments: ",len(model_yeast.compartments))

When using Jupyter notebooks, this type of information is rendered as a table.

In [ ]:
model_yeast

Just like a regular list, objects in the `DictList` can be retrieved by index. For example, to get the 3rd reaction in the model (we use an index value of 2 because of python's 0-indexing):

In [ ]:
model_yeast.reactions[2]

Additionally, items can be retrieved by their id using the `DictList.get_by_id()` function. For example, to get the cytosolic atp metabolite object (the id is `atp_c`), we will inspect metabolites in the section 1.3
For the moment, we can focus on the reactions of our model. 

## 1.2 Reactions

We will consider the reaction glucose 6-phosphate isomerase, which interconverts glucose 6-phosphate and fructose 6-phosphate. The reaction id for this reaction in our test model is PGI. However, if you want to see the IDs of the first `N` number of reactions in the reconstruction, you can run the code below:

In [ ]:
reaction_ids = [reaction.id for reaction in model_yeast.reactions]
N=20
reaction_ids[:N]

Now, let's focus on PGI or another reaction of your choice:

In [ ]:
pgi = model_yeast.reactions.get_by_id("PGI")
pgi

We can view the full name and reaction catalyzed as strings.

In [ ]:
print(pgi.name)
print(pgi.reaction)

We can also view reaction upper and lower bounds, large numbers, typically around 1000 or more, are used as infinite limits (unconstrained fluxes). 
Because the `pgi.lower_bound` < 0, and `pgi.upper_bound` > 0, pgi is reversible.

In [ ]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

The lower and upper bound of reactions can also be modified, and the reversibility attribute will automatically be updated. The preferred method for manipulating bounds is using reaction.bounds, e.g.

In [ ]:
# Save original bounds
old_bounds = pgi.bounds

# Define and print new bounds
pgi.bounds = (0, 1000.0)
print("New bounds: ",pgi.lower_bound, "< pgi <", pgi.upper_bound)
print("Reversibility after modification:", pgi.reversibility)

# Reset bounds and show reversibility
pgi.bounds = old_bounds
print("Reversibility after resetting:", pgi.reversibility)

We can also ensure the reaction is mass balanced. This function will return elements which violate mass balance. If it comes back empty, then the reaction is mass balanced.

In [ ]:
pgi.check_mass_balance()

In order to add a metabolite, we pass in a dictionary with the metabolite object and its coefficient

In [ ]:
pgi.add_metabolites({model_yeast.metabolites.get_by_id("h_c"): -1})
pgi.reaction

The reaction is no longer mass balanced

In [ ]:
pgi.check_mass_balance()

We can remove the metabolite, and the reaction will be balanced once again.



In [ ]:
pgi.subtract_metabolites({model_yeast.metabolites.get_by_id("h_c"): -1})
print(pgi.reaction)
print(pgi.check_mass_balance())

## 1.3 Metabolites

We will consider cytosolic atp as our metabolite, which has the id `atp_c` in our test model. However, if you want to see the IDs of the first N metabolites in the reconstruction, you can run the code below:

In [ ]:
metabolite_ids = [metabolite.id for metabolite in model_yeast.metabolites]
N=20
metabolite_ids[:N]

Now, let's focus on `atp_c` or another metabolite of your choice:

In [ ]:
atp = model_yeast.metabolites.get_by_id("atp_c")
atp

We can print out the metabolite name and compartment (cytosol in this case) directly as string.



In [ ]:
print(atp.name)
print(atp.compartment)

We can see that ATP is a charged molecule in our model.



In [ ]:
atp.charge

We can see the chemical formula for the metabolite as well.



In [ ]:
print(atp.formula)

## 1.4 Genes

The `gene_reaction_rule` is a boolean representation of the gene requirements for this reaction to be active as described in <a href="https://www.nature.com/articles/nprot.2011.308">Schellenberger et al 2011 Nature Protocols 6(9):1290-307</a>.

The gene-protein-reaction rules (GPR) are stored as `GPR class` in the GPR field of a reaction object. A string representation can be extracted using `gene_reaction_rule` on a Reaction object.

In [ ]:
gpr_string = pgi.gene_reaction_rule
print(gpr_string)

Corresponding gene objects also exist. These objects are tracked by the reactions itself, as well as by the model

In [ ]:
pgi_gene = model_yeast.genes.get_by_id("YBR196C")
pgi_gene

To check that this gene is also on KEGG database to assess the consistency of the metabolic reconstruction, click <a href="https://www.genome.jp/entry/sce:YBR196C"> here</a>. 

## 1.5 Running FBA

In the code chunks below we are running FBA using the `optimize()` function. We can also see import and export fluxes in our FBA solution using the `summary()` function.

In [ ]:
# Run FBA
model_yeast.optimize()

# Show exchange fluxes
model_yeast.summary()

We can also inspect all reactions that consume or produce a given metabolite in our FBA solution. For example, let't have a look at ATP:

In [ ]:
model_yeast.metabolites.atp_c.summary()

To inspect the flux of a specific reaction of interest:

In [ ]:
model_yeast.reactions.PGK.summary()

To inspect fluxes through the first N reactions:

In [ ]:
solution = model_yeast.optimize()
solution.fluxes[:N]

## 1.6 Simulating Knockouts

The `knock_out()` function can be used to knockout reactions or genes. In the case of reaction knockouts, the function will simply set both lower and upper bounds to 0, therefore preventing any flux through the reaction. For gene knockouts, it will evaluate the GPR first and only set the upper and lower bounds to 0 if there are no alternative genes that catalyze the associated reaction(s).

In [ ]:
model_yeast=cobra.io.read_sbml_model("iMM904.xml.gz")
pgi=model_yeast.reactions.get_by_id("PGI")
print("before KO: %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound))


gene=model_yeast.genes.get_by_id("YBR196C")
gene.knock_out()
print("after KO: %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound))


One may often want to make small changes to a model and evaluate their impacts. For example, we may want to knock-out all reactions sequentially, and see what the impact of this is on the objective function. One way of doing this would be to create a new copy of the model before each knock-out with the `model.copy()` function. However, even with small models, this is a very slow approach as models are quite complex objects. Instead, it is better to carry out the knock-out, optimize (i.e. solve FBA problem), and then manually reset the reaction bounds before proceeding with the next reaction. Since this is such a common scenario, `cobrapy` allows us to use the model as a context, to have changes reverted automatically.

Here we knock out the first N reactions and check for new growth rate values:

In [ ]:
# Import the model again to reverse the previous edits
model_yeast=cobra.io.read_sbml_model("iMM904.xml.gz")

# Ensure that glpk solver is used
model_yeast.solver = 'glpk'

# Show FBA solution growth rate prior to reaction knockouts
model_yeast.optimize()
print("Pre reaction knockout growth rate: ",model_yeast.objective.value)

# Define first N number of reactions to knock out
N=20

# Simulate knockouts of N single reactions 
for reaction in model_yeast.reactions[:N]:
    with model_yeast as model_yeast:  # Prevent editing of the original model
        reaction.knock_out()
        model_yeast.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model_yeast.objective.value))

Next we will knock out genes instead of reactions.

In [ ]:
# Import the model again to reverse the previous edits
model_yeast=cobra.io.read_sbml_model("iMM904.xml.gz")

# Ensure that glpk solver is used
model_yeast.solver = 'glpk'

# Show FBA solution growth rate prior to gene knockouts
model_yeast.optimize()
print("Pre gene knockout growth rate: ",model_yeast.objective.value)

# Define first N number of genes to knock out
N=20

# Simulate knockouts of N genes and print values
for gene in model_yeast.genes[:N]:
    with model_yeast as model_yeast:  # Prevent editing the original model
        gene.knock_out()
        model_yeast.optimize()
        print('%s, new growth rate %f' %
              (gene.id, model_yeast.objective.value))
        

# Simulate gene knockouts for all genes, this time we are storing the results in a vector for plotting
genes_ids=[gene.id for gene in model_yeast.genes]
grow_rates=[]
for gene in model_yeast.genes:
    with model_yeast as model_yeast:  # Prevent editing the original model
        gene.knock_out()
        model_yeast.optimize()
        grow_rates.append(model_yeast.objective.value)

Let's see the distribution of our results creating a histogram plot with the vectors obtained above:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.array(genes_ids)
y = np.array(grow_rates)

plt.hist(y)
plt.xlabel('Growth rates')
plt.ylabel('Number of genes')
plt.savefig("distribution.png", dpi=100, bbox_inches='tight',pad_inches=0)

Let's try a different visualization, this time we will create a big scatter plot showing the growth rate of each gene-knockout.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.array(genes_ids)
y = np.array(grow_rates)

fig = plt.figure(figsize=(340, 10))
plt.scatter(x, y)
plt.margins(x=0.001) 
plt.xticks(rotation=90)
plt.savefig("scatter.png", dpi=100, bbox_inches='tight',pad_inches=0)

Since the generate plot is very wide, it will appear as a blank box. Double click on it to expand it, then you can scroll to the right to inspect gene-knockout-induced changes in growth rate.

### Questions

1. The distribution of predicted growth rates appears to be bimodal, with a small peak on the left and a larger peak on the right, can you explain why it has this shape? 
2. Can you verify the consistency between gene and reactions knockouts results using a gene or a reaction of your choice?
3. Can you verify the essentiality of your gene of choice from the previous exercise using relevant databases (e.g. KEGG and the SGD)?
4. Do you expect these results to change if we change the medium where we are growing our yeast model? 

### Solutions 
#### Question 1

Answer here

#### Question 2

In [ ]:
# Code here

In [ ]:
# Code here

#### Question 3

Answer here

#### Question 4

Answer here